In [ ]:
#TabulaRazr - specific to calculate  - TABLE Parser
#Infers a table with arbitrary number of columns from reoccuring patterns in text lines
#(c) Alexander Hirner 2016, no redistribution without permission
#Contributions: ____ (refactoring), UI styling (), ....


#Main assumptions Table identificatin:
#1) each row is either in one line or not a row at all
#2) each column features at least one number (=dollar amount)
#2a) each column features at least one date-like string [for time-series only]
#3) a table exists if rows are in narrow consecutive order and share similarities --> scoring algo [DONE] 
#4) each column is separated by more than x consecutive whitespace indicators (e.g. '  ' or '..')

#Feature List Todo:
#1) Acknowledge footnotes / make lower meta-data available
#2) make delimiter length smartly dependent on number of columns (possible iterative approach)
#3) improve captioning: expand non canonical values in tables [DONE] .. but not to the extent how types match up  --> use this to further
## delineate between caption and headers
#4) UI: parameterize extraction on the show page on the fly
#5) deeper type inference on token level: type complex [DONE], subtype header (centered, capitalized), 
## subtype page nr., type free flow [DONE, need paragraph]
#6) Respect negative values with potential '-' for numerical values
#7) 
#8) classify tables with keywords (Muni Bonds) and unsupervised clustering (Hackathon)
#9) Restructure folder and URI around MD5 hash (http://stackoverflow.com/questions/24570066/calculate-md5-from-werkzeug-datastructures-filestorage-without-saving-the-object)

In [1]:
from backend import *
from data_query import *
import os

In [79]:
from fuzzywuzzy import fuzz
#print (fuzz.partial_ratio("this is a test!", "this text is"))
#print (fuzz.partial_ratio("USES OF FUNDS", "USES from FUNDS"))
#print (fuzzy_str_match("aaaaaaaaaa text this", "this test is"))

print (fuzzy_str_match("USES OF FUNDS", "USES OF FUNDS"))
print (fuzzy_str_match("USES OF FUNDS", "USES of FUNDS"))
print (fuzzy_str_match("USES OF FUNDS", "uses of funds"))
print (fuzzy_str_match("USES OF FUNDS", "USES which include other FUNDS"))


print (fuzzy_str_match("USES OF FUNDS", "Note 5 - Assets Limited as to Use"))
print (fuzzy_str_match("USES OF FUNDS", "FINANCIAL STATEMENT AMOUNTS - Continued"))

print (fuzzy_str_match("USES OF FUNDS", "FUNDS OF USES"))


print (fuzzy_str_match("USES OF FUNDS", "TABLE OF CONTENTS"))

print (fuzzy_str_match("Debt Service", "Total Periodic Debt Service"))

#print (fuzzy_str_match("USES OF FUNDS", "Note 5 - Assets Limited as to Use"))
#print (fuzzy_str_match("USES OF FUNDS", "FINANCIAL STATEMENT AMOUNTS - Continued"))


('fuzzy_partial of USES OF FUNDS vs USES OF FUNDS * penalty 1.00', 1.0)
1.0
('fuzzy_partial of USES OF FUNDS vs USES of FUNDS * penalty 1.00', 0.85)
0.9334
('fuzzy_partial of USES OF FUNDS vs uses of funds * penalty 1.00', 0.15)
('fuzzy_partial lower_case of USES OF FUNDS vs uses of funds * penalty 1.00', 1.0)
0.667
('fuzzy_partial of USES OF FUNDS vs USES which include other FUNDS * penalty 1.00', 0.54)
('fuzzy_partial lower_case of USES OF FUNDS vs USES which include other FUNDS * penalty 1.00', 0.69)
('fuzzy_partial token_sort_lower_case of USES OF FUNDS vs USES which include other FUNDS * penalty 1.00', 0.62)
0.16528
('fuzzy_partial of USES OF FUNDS vs Note 5 - Assets Limited as to Use * penalty 1.00', 0.23)
('fuzzy_partial lower_case of USES OF FUNDS vs Note 5 - Assets Limited as to Use * penalty 1.00', 0.54)
('fuzzy_partial token_sort_lower_case of USES OF FUNDS vs Note 5 - Assets Limited as to Use * penalty 1.00', 0.56)
0.13864
('fuzzy_partial of USES OF FUNDS vs FINANCIAL STATE

In [64]:
print (fuzzy_str_match("Maturity", "ii"))
#print (fuzz.partial_ratio("Maturity", "i"))
print (fuzzy_str_match("Maturity", "Matarit"))
print (fuzzy_str_match("Maturity", "i"))
print (fuzzy_str_match("Maturity", "     i   "))


print ("Maturity".lower() in "i".lower())

('fuzzy_partial of Maturity vs ii * penalty 0.25', 0.125)
('fuzzy_partial lower_case of Maturity vs ii * penalty 0.25', 0.125)
('fuzzy_partial token_sort_lower_case of Maturity vs ii * penalty 0.25', 0.125)
None
('fuzzy_partial of Maturity vs Matarit * penalty 0.88', 0.7525)
0.89011
('fuzzy_partial of Maturity vs i * penalty 0.12', 0.125)
('fuzzy_partial lower_case of Maturity vs i * penalty 0.12', 0.125)
('fuzzy_partial token_sort_lower_case of Maturity vs i * penalty 0.12', 0.125)
None
('fuzzy_partial of Maturity vs      i    * penalty 0.12', 0.01625)
('fuzzy_partial lower_case of Maturity vs      i    * penalty 0.12', 0.01625)
('fuzzy_partial token_sort_lower_case of Maturity vs      i    * penalty 0.12', 0.01625)
None
False
